# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# Make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create a Keyspace (collection of tables)
try: 
    session.execute(""" CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    {'class': 'SimpleStrategy', 'replication_factor': 1}""")
except Exception as e: 
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try: 
    session.set_keyspace('udacity')
except Exception as e: 
    print(e)
    

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
## Primary key was with sessionID and itemInsession since that is what this query wants to be filtered by. 

## Data type info: https://cassandra.apache.org/doc/latest/cassandra/cql/types.html 
## 

query = "CREATE TABLE IF NOT EXISTS song_info"
query = query + "( sessionId int, itemInSession int, artist text , song text , length float, PRIMARY KEY (sessionId, itemInSession))"

try: 
    session.execute(query)
except Exception as e: 
    print(e)
                    

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_info ( sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
    
        ## line[x] x is the position in the extracted csv file
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Test the query 1
query = "Select artist, song, length FROM song_info WHERE sessionID = 338 AND itemInSession = 4  "

try: 
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.length)

#answer: Faithless Music Matters (Mark Knight Dub) 495.3073

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

## Partition key was given as sessionId and user_id since this is what the query wants to be filtered on. Then to sort on itemInSession I added it as a cluster column. 

query = "CREATE TABLE IF NOT EXISTS user_library"
query = query + "(sessionId int, user_id int, artist text , song text, user_first text, user_last text, itemInSession int, PRIMARY KEY ((sessionId, user_id), itemInSession))"

try: 
    session.execute(query)
except Exception as e: 
    print(e)
                    

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements
        query = "INSERT INTO user_library (sessionId, user_id, artist, song, user_first, user_last, itemInSession)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"

        ## line[x] x is the position in the extracted csv file
        session.execute(query, (int(line[8]), int(line[10]), line[0],line[9], line[1], line[4], int(line[3])))

In [13]:
## Test Query 2
query = "Select artist, song, user_first, user_last FROM user_library WHERE sessionId = 182 AND user_id = 10 "

try: 
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.artist, row.song, row.user_first, row.user_last)
#answer: Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## The query wants to filter on song so that is part of the partition key. 
##I also, added user_id as a secondary key just incase the song's weren't unique. This way you could search by both

query = "CREATE TABLE IF NOT EXISTS user_songs"
query = query + "(song text, user_first text, user_last text, user_id int , PRIMARY KEY (song, user_id))"

try: 
    session.execute(query)
except Exception as e: 
    print(e)
                    

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_songs (song, user_first, user_last, user_id)"
        query = query + "VALUES(%s, %s, %s, %s)"
 
        ##line[x] x is the position in the extracted csv file
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [16]:
## Test Query 3
query = "SELECT user_first, user_last FROM user_songs WHERE song = 'All Hands Against His Own'  "

try: 
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print (row.user_first, row.user_last)

#answer: Jacqueline Lynch, Tegan levine, Sara Johnson

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
## DROP TABLES
# I wrote the drops individually so you could test and edit each data mondel and drop it independently 

In [18]:
query = "drop table song_info"
try: 
    rows = session.execute(query)
except Exception as e: 
    print(e)
    


In [19]:
query = "drop table user_library"
try: 
    rows = session.execute(query)
except Exception as e: 
    print(e)

In [20]:
query = "drop table user_songs"
try: 
    rows = session.execute(query)
except Exception as e: 
    print(e)

### Close the session and cluster connection¶

In [21]:
session.shutdown()
cluster.shutdown()